In [ ]:

# 고대 문명 유적 탐사
# https://www.codetree.ai/training-field/frequent-problems/problems/ancient-ruin-exploration/description?page=1&pageSize=20



### K턴 진행

## 가능한 모든 방법으로 해보아서 1차 유물 가치 계산
    # 유물 가치가 없으면 게임 종료
    # 만약 가치가 동일하면 (2), (3)에 따라서 하나 정하기,
    # 유물 조각 사라지기, 유물 가치 저장

# 1. 조각 채우기
    # 작은 열부터, 큰 행부터
# 2. 3개 이상 묶음 있는지 확인
    # 없으면 해당 턴 종료
    # 있으면 없애고 1번으로 가기


# 답은 각 턴의 유물 가치 합계를 순서대로 출력







K, M = list(map(int, input().split()))

mat = []
for i in range(5):
    mat.append(list(map(int, input().split())))

jogak = list(map(int, input().split()))
jogak_idx = 0


# for i in mat:
    # print(i)
# print(jogak)

dys = [0,1,0,-1]
dxs = [1,0,-1,0]

def remove(removeList):
    global mat

    # mat에서 유물 제거

    for each in removeList:
        y,x = each
        mat[y][x] = 0




def in_range(y,x):
    return 0<=y and y<5 and 0<=x and x<5

def bfs(temMat):

    # bfs 진행
    totalCnt = 0
    removeList = []
    visited = [[False for j in range(5)] for i in range(5)]
    for r_idx in range(5):
        for c_idx in range(5):

            queue = []

            # 시작점 지정
            if visited[r_idx][c_idx] == True:
                continue

            queue.append([r_idx, c_idx])
            visited[r_idx][c_idx] = True

            curNum = temMat[r_idx][c_idx]
            cnt = 1
            temRemoveList = [[r_idx, c_idx]]

            # 각 시작점에서 bfs 진행
            while queue:
                cy, cx = queue.pop(0)

                for dy,dx in zip(dys, dxs):
                    ny, nx = cy+dy, cx+dx

                    if(not in_range(ny,nx) or curNum != temMat[ny][nx] or visited[ny][nx] == True):
                        continue

                    queue.append([ny,nx])
                    visited[ny][nx] = True
                    cnt += 1
                    temRemoveList.append([ny,nx])

            if(cnt >= 3):
                totalCnt += cnt
                removeList += temRemoveList

    return totalCnt, removeList

def checkPackage(r, c, rotMat):

    ### 제거할 r,c 담은 리스트 , 점수 리턴


    temMat = [[mat[i][j] for j in range(5)] for i in range(5)]

    # rotMat을 mat에 넣어주기
    r_idx = 0
    for i in range(r-1,r+2):
        c_idx = 0
        for j in range(c-1,c+2):
            temMat[i][j] = rotMat[r_idx][c_idx]
            c_idx += 1
        r_idx += 1

    totalCnt, removeList = bfs(temMat[:])

    return removeList, totalCnt




def rotate90(r,c):

    temMat = [[mat[i][j] for j in range(c-1,c+2)] for i in range(r-1,r+2)]

    rotMat = list(map(list, zip(*temMat[::-1])))
    # 1 2 3
    # 4 5 6
    # 7 8 9

    return rotMat

def rotate180(r,c):

    temMat = [[mat[i][j] for j in range(c-1,c+2)] for i in range(r-1,r+2)]

    rotMat = [i[::-1] for i in temMat[::-1]]
    return rotMat

def rotate270(r,c):

    temMat = [[mat[i][j] for j in range(c-1,c+2)] for i in range(r-1,r+2)]

    rotMat = list(map(list, zip(*temMat)))[::-1]
    return rotMat



def first_process():
    global mat

    # 1차 유물 가치 계산, mat 갱신(유물 지우기), score 리턴

    totalList = []

    # for 문으로 r,c 3개 3개 바꿔가며 반복
        # 세 각도에 대해 회전
            # removeList, score = checkPackage()
            # totalList에 [score, 회전한 각도, 회전 중심 좌표의 열, 행, removeList] 저장

    for r in range(1,4):
        for c in range(1,4):
            rotMat = rotate90(r,c)
            removeList, score = checkPackage(r, c, rotMat)
            totalList.append([score, 90, c,r, removeList[:]])

            rotMat = rotate180(r,c)
            removeList, score = checkPackage(r, c, rotMat)
            totalList.append([score, 180, c,r, removeList[:]])

            rotMat = rotate270(r,c)
            removeList, score = checkPackage(r, c, rotMat)
            totalList.append([score, 270, c,r, removeList[:]])


    # totalList sort
    totalList.sort(key=lambda x : (-1*x[0], x[1], x[2], x[3]))
    # print(totalList)

    target = totalList[0]
    score, angle, c, r, rmList = target

    # 첫 번째 원소의 score가 0이면 return 0
    if(score == 0):
        return 0, []



    ## 실제 회전 반영
    rotMat = []
    if(angle == 90):
        rotMat = rotate90(r,c)
    elif(angle == 180):
        rotMat = rotate180(r,c)
    elif(angle == 270):
        rotMat = rotate270(r,c)

    # rotMat을 mat에 넣어주기
    r_idx = 0
    for i in range(r-1,r+2):
        c_idx = 0
        for j in range(c-1,c+2):
            mat[i][j] = rotMat[r_idx][c_idx]
            c_idx += 1
        r_idx += 1

    # remove
    remove(rmList[:])


    return score, rmList



def fillJogak(removeList):
    global jogak_idx

    removeList.sort(key=lambda x:(x[1],-1*x[0]))

    for each in removeList:
        y,x = each

        mat[y][x] = jogak[jogak_idx]
        jogak_idx += 1




scoreList = [] # 각 턴 점수 저장
for round in range(K):

    round_score = 0

    # 1차 유물 가치 계산
    score, removeList = first_process()

    if(score == 0):
        break # 끝내기

    round_score += score

    while True:
        # 조각 채우기
        fillJogak(removeList[:])

        # for i in mat:
        #     print(i)
        # print()

        # print(removeList)



        # 유물 제거
        temMat = [[mat[i][j] for j in range(5)] for i in range(5)]
        score, removeList = bfs(temMat)
        if(len(removeList) == 0):
            break

        round_score += score

        remove(removeList[:])

    scoreList.append(round_score)

print(" ".join(list(map(str, scoreList))))
